# Calculo de Probabilidade de Viagem

Este notebook calcula a probabilidade de uma pessoa infectada viajar para um determinadas cidade a qual possui conexão.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_municipio = pd.read_csv("data/integrado/municipio.csv")
df_fluxo_aereo = pd.read_csv("data/integrado/fluxo_aereo.csv")
df_fluxo_rodoviario = pd.read_csv("data/integrado/fluxo_rodoviario.csv")

Funções de extração dos dados

In [37]:
def info_cidade(codigo_municipio, df_municipio, df_fluxo_aereo, df_fluxo_rodoviario):
    
    #Consultando valores nos dataframe a partir do código da cidade
    total_populacao = df_municipio[df_municipio.cod_mun == codigo_municipio].populacao_2021
    
    df_aereo = df_fluxo_aereo[df_fluxo_aereo.cod_mun_origem == codigo_municipio][['cod_mun_destino', 'total_passagens_2020']].rename(columns={"cod_mun_destino": "cod_mun"})
    
    
    df_rod_1 = df_fluxo_rodoviario[(df_fluxo_rodoviario.cod_mun_a == codigo_municipio)][['cod_mun_b', 'freq_saida_2016']].rename(columns={"cod_mun_b": "cod_mun"})
    df_rod_2 = df_fluxo_rodoviario[(df_fluxo_rodoviario.cod_mun_b == codigo_municipio)][['cod_mun_a', 'freq_saida_2016']].rename(columns={"cod_mun_a": "cod_mun"})
    df_rod = df_rod_1.append(df_rod_2, ignore_index=True).sort_values(by=['freq_saida_2016'])
    
    df_fluxo = df_aereo.merge(df_rod[['cod_mun', 'freq_saida_2016']], how='outer', left_on='cod_mun', right_on='cod_mun')
    df_fluxo = df_aereo #TODO: Remover esta linha quando adicionar o rodoviario -> Bug do código 0
    df_fluxo.fillna(0, inplace = True)

    df_fluxo = df_fluxo.merge(df_municipio[['cod_mun', 'populacao_2021']], how='left', left_on='cod_mun', right_on='cod_mun')
    
    lista_fluxo = list(df_fluxo.itertuples(index=False, name=None))
    
    return (total_populacao, lista_fluxo)

Calculo da Probabilidade

In [40]:
FATOR_RODOV = 20 *365

## Calculando as probabilidade de viajar e estar infectado
total_populacao, lista_fluxo= info_cidade(2611606, df_municipio, df_fluxo_aereo, df_fluxo_rodoviario)
fluxo_total = sum(j for i, j, k in lista_fluxo)
fluxo_total += sum(k for i, j, k in lista_fluxo) * FATOR_RODOV ##Valor que tem que mudar, essa é minha estimativa

numero_infectados = 10
prob_infectado = numero_infectados / total_populacao

prob_final = []
for codigo, fluxo_aereo, fluxo_rodo in lista_fluxo:
    # print(f"{codigo} - {fluxo_aereo} - {fluxo_rodo}")

    fluxo = fluxo_aereo #+ fluxo_rodo * FATOR_RODOV
    prob_viajar = fluxo / fluxo_total

    prob_final.append((codigo, prob_viajar))

In [41]:
prob_final.sort(key=lambda x:-x[1])
for cod_mun, prob in prob_final:
    nome = df_municipio.loc[df_municipio.cod_mun==cod_mun, 'nome_mun'].values[0]
    print(f"{nome} - {prob}")

São Paulo - 5.891155591278939e-07
Rio de Janeiro - 1.9969813711772068e-07
Brasília - 1.8229774444683982e-07
Fortaleza - 1.4628603298148535e-07
Salvador - 8.608658530165445e-08
Manaus - 6.466345499985517e-08
Porto Alegre - 6.16505066109836e-08
Belo Horizonte - 4.537278612587438e-08
Campinas - 4.430552920338582e-08
Fernando de Noronha - 4.18364713615517e-08
Curitiba - 4.1272056642927936e-08
Belém - 4.0126807868411365e-08
Goiânia - 3.807233829262088e-08
Cuiabá - 3.615948550077599e-08
Petrolina - 2.61601091039216e-08
Vitória - 2.590355695909262e-08
São Luís - 2.5624428225518687e-08
Teresina - 2.460642931483729e-08
Florianópolis - 2.3863454303412558e-08
Itajaí - 2.225025441672792e-08
Campo Grande - 2.048928049462179e-08
Aracaju - 1.3443332389038638e-08
Porto Velho - 1.333455427963115e-08
Uberlândia - 1.186091875973348e-08
Foz do Iguaçu - 1.162899562080808e-08
Juazeiro do Norte - 9.31386906587135e-09
Palmas - 7.3004478332535015e-09
Maringá - 6.625202588063622e-09
Macapá - 5.299341103587445e-

0.05402638218731237